## Delta lake data skipping and partitioning

In [0]:
%sql
drop table if exists orders_managed;

create table orders_managed (
  order_id long,
  sku string,
  product_name string,
  category string,
  quantity long,
  price double,
  country string
)
using delta

In [0]:
%sql
INSERT INTO orders_managed VALUES
(1, 'SKU-001', 'Widget A', 'widgets', 2, 199.99, 'IN'),
(2, 'SKU-002', 'Widget B', 'widgets', 1, 149.50, 'US'),
(3, 'SKU-003', 'Gadget C', 'gadgets', 3, 249.00, 'UK'),
(4, 'SKU-004', 'Tool D', 'tools', 5, 89.99, 'CA'),
(5, 'SKU-005', 'Item E', 'supplies', 4, 59.49, 'AU'),
(6, 'SKU-006', 'Widget A', 'widgets', 2, 199.99, 'DE'),
(7, 'SKU-007', 'Widget B', 'widgets', 7, 175.25, 'FR'),
(8, 'SKU-008', 'Gadget C', 'gadgets', 1, 300.75, 'SG'),
(9, 'SKU-009', 'Tool D', 'tools', 6, 120.00, 'JP'),
(10,'SKU-010', 'Item E', 'supplies', 3, 49.99, 'BR'),

(11,'SKU-011', 'Widget A', 'widgets', 2, 199.99, 'IN'),
(12,'SKU-012', 'Widget B', 'widgets', 4, 130.00, 'US'),
(13,'SKU-013', 'Gadget C', 'gadgets', 2, 275.45, 'UK'),
(14,'SKU-014', 'Tool D', 'tools', 8, 95.75, 'CA'),
(15,'SKU-015', 'Item E', 'supplies', 3, 65.20, 'AU'),
(16,'SKU-016', 'Widget A', 'widgets', 1, 189.99, 'DE'),
(17,'SKU-017', 'Widget B', 'widgets', 9, 210.50, 'FR'),
(18,'SKU-018', 'Gadget C', 'gadgets', 2, 310.00, 'SG'),
(19,'SKU-019', 'Tool D', 'tools', 4, 105.40, 'JP'),
(20,'SKU-020', 'Item E', 'supplies', 5, 79.99, 'BR'),

(21,'SKU-021', 'Widget A', 'widgets', 3, 205.75, 'IN'),
(22,'SKU-022', 'Widget B', 'widgets', 2, 160.99, 'US'),
(23,'SKU-023', 'Gadget C', 'gadgets', 4, 298.20, 'UK'),
(24,'SKU-024', 'Tool D', 'tools', 6, 140.00, 'CA'),
(25,'SKU-025', 'Item E', 'supplies', 7, 60.00, 'AU'),
(26,'SKU-026', 'Widget A', 'widgets', 3, 209.10, 'DE'),
(27,'SKU-027', 'Widget B', 'widgets', 5, 180.00, 'FR'),
(28,'SKU-028', 'Gadget C', 'gadgets', 6, 333.33, 'SG'),
(29,'SKU-029', 'Tool D', 'tools', 2, 99.99, 'JP'),
(30,'SKU-030', 'Item E', 'supplies', 4, 49.49, 'BR'),

(31,'SKU-031', 'Widget A', 'widgets', 8, 210.00, 'IN'),
(32,'SKU-032', 'Widget B', 'widgets', 3, 170.75, 'US'),
(33,'SKU-033', 'Gadget C', 'gadgets', 2, 260.10, 'UK'),
(34,'SKU-034', 'Tool D', 'tools', 5, 130.30, 'CA'),
(35,'SKU-035', 'Item E', 'supplies', 1, 55.55, 'AU'),
(36,'SKU-036', 'Widget A', 'widgets', 4, 205.99, 'DE'),
(37,'SKU-037', 'Widget B', 'widgets', 6, 195.60, 'FR'),
(38,'SKU-038', 'Gadget C', 'gadgets', 3, 340.25, 'SG'),
(39,'SKU-039', 'Tool D', 'tools', 7, 115.49, 'JP'),
(40,'SKU-040', 'Item E', 'supplies', 3, 65.99, 'BR'),

(41,'SKU-041', 'Widget A', 'widgets', 2, 222.22, 'IN'),
(42,'SKU-042', 'Widget B', 'widgets', 5, 155.40, 'US'),
(43,'SKU-043', 'Gadget C', 'gadgets', 1, 288.80, 'UK'),
(44,'SKU-044', 'Tool D', 'tools', 3, 135.50, 'CA'),
(45,'SKU-045', 'Item E', 'supplies', 6, 45.90, 'AU'),
(46,'SKU-046', 'Widget A', 'widgets', 1, 199.10, 'DE'),
(47,'SKU-047', 'Widget B', 'widgets', 4, 175.20, 'FR'),
(48,'SKU-048', 'Gadget C', 'gadgets', 5, 320.60, 'SG'),
(49,'SKU-049', 'Tool D', 'tools', 8, 129.49, 'JP'),
(50,'SKU-050', 'Item E', 'supplies', 2, 59.99, 'BR');


In [0]:
%sql
select * from orders_managed

In [0]:
dbutils.fs.mkdirs("/Volumes/workspace/default/orders_volume/partitionDemo")

In [0]:
orders_df = spark.read.table("orders_managed")

In [0]:
orders_df.show()

In [0]:
orders_df.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/orders_volume/partitionDemo")

In [0]:
display(dbutils.fs.ls("/Volumes/workspace/default/orders_volume/partitionDemo"))

In [0]:
%sql
desc history delta.`/Volumes/workspace/default/orders_volume/partitionDemo`

In [0]:
%sql
select * from JSON.`dbfs:/Volumes/workspace/default/orders_volume/partitionDemo/_delta_log/00000000000000000000.json`

In [0]:
%sql
drop table if exists orders_managed_partitioned;

create table orders_managed_partitioned (
  order_id long,
  sku string,
  product_name string,
  category string,
  quantity long,
  price double,
  country string
)
using delta
partitioned by (country)

In [0]:
%sql
insert into orders_managed_partitioned
select * from orders_managed

In [0]:
%sql
select * from orders_managed_partitioned;

In [0]:
partitioned_df = spark.read.table("orders_managed_partitioned")

In [0]:
dbutils.fs.mkdirs("/Volumes/workspace/default/orders_volume/partitionedTable")

In [0]:
partitioned_df.write.partitionBy("country").format("delta").mode("append").save("/Volumes/workspace/default/orders_volume/partitionedTable")

In [0]:
%sql
select * from delta.`/Volumes/workspace/default/orders_volume/partitionedTable`

In [0]:
display(dbutils.fs.ls("/Volumes/workspace/default/orders_volume/partitionedTable"))

In [0]:
%sql
desc detail delta.`/Volumes/workspace/default/orders_volume/partitionedTable`

In [0]:
%sql
show partitions orders_managed_partitioned
  

In [0]:
orders_df.printSchema()

In [0]:
%sql
select category, sum(quantity) from orders_managed_partitioned
where country="IN"
group by category;

## seelct count(*) will not read any files. it will get the result from the metadat it self 

In [0]:
%sql
select count(*) from orders_managed_partitioned;

In [0]:
%sql
select sum(quantity) as total_qty 
from orders_managed_partitioned
where price>100.00;